# Python Dash App Development

In this notebook we are going to create a simple dash app in order to provide an interface for an user.

App will be pretty simple and will allow the user to upload or drag and drop and image to inference

In [1]:
import dash
from dash import Dash, dcc, html, Input, Output, State, callback
from dash.dependencies import Input, Output, State
import datetime
import requests
import json
import base64
import io
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random
from PIL import Image
import numpy as np


app = dash.Dash(__name__, external_stylesheets=['https://codepen.io/chriddyp/pen/bWLwgP.css'])


In [2]:
API_ENDPOINT = "https://grv4d32y32.execute-api.us-east-2.amazonaws.com/production_airplane/detect-airplane"

### Functions

In [3]:
# Function to plot
def draw_bounding_boxes_local(image_file, detections, threshold=0.3):
    image = np.array(Image.open(image_file))
    
    fig, ax = plt.subplots(figsize=(10, 10))
    ax.imshow(image)
    
    image_height, image_width = image.shape[:2]
    assigned_colors = dict()
    detected_count = 0
    
    for detection in detections:
        klass, score, x0, y0, x1, y1 = detection
        if score < threshold:
            continue
        
        detected_count += 1
        class_id = int(klass)
        
        if class_id not in assigned_colors:
            assigned_colors[class_id] = random.random(), random.random(), random.random()
            
        xmin = int(x0 * image_width)
        ymin = int(y0 * image_height)
        xmax = int(x1 * image_width)
        ymax = int(y1 * image_height)
        
        bounding_box = plt.Rectangle(
            (xmin, ymin),
            xmax - xmin,
            ymax - ymin,
            fill=False,
            edgecolor=assigned_colors[class_id],
            linewidth=2.5
        )
        
        ax.add_patch(bounding_box)
        ax.text(
            xmin,
            ymin - 2,
            "{:.3f}".format(score),
            bbox=dict(facecolor=(1, 0, 0, 1), alpha=0.5),
            color="white"
        )
    ax.axis('off')  

    if detected_count > 1:
        print(f"Model detected {detected_count} airplanes - Threshold: {threshold}")
    elif detected_count == 0:
        print(f"{detected_count} airplane detected - Threshold: {threshold}")
    else:
        print(f"Model detected only {detected_count} airplane - Threshold: {threshold}")

    buf = io.BytesIO()
    plt.tight_layout()
    plt.savefig(buf, format="JPEG", bbox_inches='tight', pad_inches=0)
    plt.close()

    image_base64 = base64.b64encode(buf.getvalue()).decode()
    buf.close()

    return image_base64


    
    
# Function to get predictions from API
def get_detections_from_api(encoded_image_data, api_url):
    headers = {'Content-Type': 'application/jpeg'}
    response = requests.post(api_url, data=encoded_image_data, headers=headers)
    if response.status_code == 200:
        body_content_string = response.json()['body']
        actual_detections_content = json.loads(body_content_string)
        actual_detections = actual_detections_content['detections']
        return actual_detections
    else:
        print(f"Request failed with status code {response.status_code}: {response.text}")
        return None 


# Send image for prediction and display bounding boxes
def parse_contents(contents, filename, date):
    content_type, content_string = contents.split(',')
    decoded = base64.b64decode(content_string)
    detections = get_detections_from_api(decoded, API_ENDPOINT)
    
    buffer = io.BytesIO(decoded)
    buffer.seek(0)
    
    image_base64 = draw_bounding_boxes_local(buffer, detections)
    
    return html.Div([
        html.H5(filename),
        html.H6(datetime.datetime.fromtimestamp(date)),
        html.Img(src=f"data:image/jpeg;base64,{image_base64}", style={'maxWidth': '100%', 'margin': 'auto'}),
        html.Hr(),
    ])




In [4]:
# Layout
app.layout = html.Div([
    dcc.Upload(
        id='upload-image',
        children=html.Div([
            'Drag and Drop or ',
            html.A('Select Files')
        ]),
        style={
            'width': '100%',
            'height': '60px',
            'lineHeight': '60px',
            'borderWidth': '1px',
            'borderStyle': 'dashed',
            'borderRadius': '5px',
            'textAlign': 'center',
            'margin': '10px',
            'backgroundColor': '#f7f7f7'
        },
        multiple=False
    ),
    html.Div(id='output-image-upload', style={'textAlign': 'center'}),

], style={
    'fontFamily': 'Arial, sans-serif',
    'background': '#f1f1f1',
    'height': '100vh',
    'padding': '20px'
})

@app.callback(Output('output-image-upload', 'children'),
              Input('upload-image', 'contents'),
              State('upload-image', 'filename'),
              State('upload-image', 'last_modified'))
def update_output(contents, name, date):
    if contents is not None:
        return parse_contents(contents, name, date)

if __name__ == '__main__':
    app.run_server(debug=True, open_browser=True)
    
# URL: http://127.0.0.1:8050/

Model detected only 1 airplane - Threshold: 0.3
Model detected 4 airplanes - Threshold: 0.3
